In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [37]:
!ls "/content/drive/My Drive/ml"


action.ipynb	 output1		UCF101
action_v2.ipynb  output_200		UCF101_test
extra		 output_500epoch_rgb	v_ApplyEyeMakeup_g01_c01.avi
n		 output_smallepoch_rgb
output		 results


In [0]:
import numpy as np
import cv2
import os
from tqdm import tqdm
import argparse
import os
import json
from keras.models import model_from_json, load_model

import numpy as np
from keras.datasets import cifar10
from keras.layers import (Activation, Conv3D, Dense, Dropout, Flatten,
                          MaxPooling3D)
from keras.layers.advanced_activations import LeakyReLU
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
import matplotlib
#import pydot
#matplotlib.use('AGG')
import matplotlib.pyplot as plt
depth=18
w=50
h=50

In [0]:
def video3d(filename):
        cap = cv2.VideoCapture(filename)
        nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        
        frames = [x * nframe / depth for x in range(depth)]
        
        framearray = []

        for i in range(depth):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frames[i])
           # wf = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  
            #hf = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
            ret, frame = cap.read()
            #print("vvvvvvv")
            #print(wf,hf)
            frame = cv2.resize(frame, (w,h))
            framearray.append(frame)#cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        cap.release()
        return np.array(framearray)
#c=video3d('/content/drive/My Drive/ml/v_ApplyEyeMakeup_g01_c01.avi')
#print(c)

In [0]:
def get_UCF_classname(filename):
        return filename[filename.find('_') + 1:filename.find('_', 2)]


In [0]:
def loaddata(video_dir,nclass, result_dir):
    files = os.listdir(video_dir)
    X = []
    labels = []
    labellist = []

    #pbar = tqdm(total=len(files))
    for v in files:
        v1 = os.path.join(video_dir, v)
        vfiles=os.listdir(v1)
        for filename in vfiles:
          #pbar.update(1)
            if filename == '.DS_Store':
                continue
            name = os.path.join(v1, filename)
            #print(files)
            label = get_UCF_classname(filename)
            if label not in labellist:
                if len(labellist) >= nclass:
                    continue
                labellist.append(label)
            labels.append(label)

            X.append(video3d(name))

      #pbar.close()
    with open(os.path.join(result_dir, 'classes.txt'), 'w') as fp:
        for i in range(len(labellist)):
            fp.write('{}\n'.format(labellist[i]))

    for num, label in enumerate(labellist):
        for i in range(len(labels)):
            if label == labels[i]:
                labels[i] = num
    #v=np.array(X)            
    #print(v.shape)
    return np.array(X).transpose((0, 2, 3, 4, 1)), labels
x,y=loaddata('/content/drive/My Drive/ml/UCF101',6,'/content/drive/My Drive/ml/results')

In [0]:
print(y)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [0]:
noofclass=6
fname_npz = 'datasett_{}_{}.npz'.format(
        noofclass,depth)#class,depth,skip
X = x.reshape((x.shape[0], w, h, depth, 3))
Y = np_utils.to_categorical(y, noofclass)

X = X.astype('float32')
np.savez(fname_npz, X=X, Y=Y)
print('Saved dataset to dataset.npz.')
print('X_shape:{}\nY_shape:{}'.format(X.shape, Y.shape))


Saved dataset to dataset.npz.
X_shape:(784, 50, 50, 18, 3)
Y_shape:(784, 6)


In [0]:
model = Sequential()
model.add(Conv3D(64, kernel_size=(3, 3, 3), input_shape=(
    X.shape[1:]), border_mode='same'))
model.add(Activation('relu'))
model.add(Conv3D(128, kernel_size=(3, 3, 3), border_mode='same'))
model.add(Activation('softmax'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), border_mode='same'))
model.add(Dropout(0.25))

model.add(Conv3D(128, kernel_size=(3, 3, 3), border_mode='same'))
model.add(Activation('relu'))
model.add(Conv3D(256, kernel_size=(3, 3, 3), border_mode='same'))
model.add(Activation('softmax'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), border_mode='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(noofclass, activation='softmax'))
#sgd = SGD(lr=0.005, momentum=0.9, nesterov=True)
model.compile(loss=categorical_crossentropy,optimizer=Adam(), metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, kernel_size=(3, 3, 3), input_shape=(50, 50, 1..., padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, kernel_size=(3, 3, 3), padding="same")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(3, 3, 3), padding="same")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, kernel_size=(3, 3, 3), padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv3D` call to th

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 50, 50, 18, 64)    5248      
_________________________________________________________________
activation_1 (Activation)    (None, 50, 50, 18, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 50, 50, 18, 128)   221312    
_________________________________________________________________
activation_2 (Activation)    (None, 50, 50, 18, 128)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 17, 17, 6, 128)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 17, 17, 6, 128)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 17, 17, 6, 128)    442496    
__________

In [0]:
pip install pydot


SyntaxError: ignored

In [0]:
#plot_model(model, show_shapes=True,to_file=os.path.join('/content/drive/My Drive/ml/output', 'model.png'))

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=43)


In [31]:
hist=model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=15,
                        epochs=150, verbose=1, shuffle=True)

Train on 627 samples, validate on 157 samples
Epoch 1/500
627/627 [==============================] - 82s 131ms/step - loss: 2.2710 - acc: 0.1914 - val_loss: 2.1602 - val_acc: 0.1465
Epoch 2/500
627/627 [==============================] - 74s 118ms/step - loss: 2.0879 - acc: 0.1340 - val_loss: 1.9690 - val_acc: 0.1401
Epoch 3/500
627/627 [==============================] - 74s 118ms/step - loss: 2.0685 - acc: 0.1643 - val_loss: 1.9800 - val_acc: 0.1465
Epoch 4/500
627/627 [==============================] - 74s 118ms/step - loss: 2.0648 - acc: 0.1722 - val_loss: 1.8677 - val_acc: 0.1465
Epoch 5/500
627/627 [==============================] - 74s 118ms/step - loss: 1.9962 - acc: 0.1738 - val_loss: 1.8360 - val_acc: 0.1975
Epoch 6/500
627/627 [==============================] - 75s 119ms/step - loss: 1.9119 - acc: 0.1978 - val_loss: 1.6117 - val_acc: 0.3121
Epoch 7/500
627/627 [==============================] - 74s 119ms/step - loss: 1.6429 - acc: 0.3461 - val_loss: 1.2845 - val_acc: 0.4586
Ep

KeyboardInterrupt: ignored

In [32]:
  model.evaluate(X_test, Y_test, verbose=0)
  model_json = model.to_json()
  if not os.path.isdir('/content/drive/My Drive/ml/output'):
      os.makedirs('/content/drive/My Drive/ml/output')
  with open(os.path.join('/content/drive/My Drive/ml/output', 'ucf101_3dcnnmodel_50_50_30_rgb.json'), 'w') as json_file:
      json_file.write(model_json)
  model.save_weights(os.path.join('/content/drive/My Drive/ml/output', 'ucf101_3dcnnmodel_50_50_30_rgb.hd5'))

  loss, acc = model.evaluate(X_test, Y_test, verbose=0)
  print('Test loss:', loss)

Test loss: 0.34954394191313704


In [33]:
print('Test accuracy:', acc)

Test accuracy: 0.8726114615513261


In [40]:

with open('/content/drive/My Drive/ml/results/classes.txt', 'r') as f:
    class_names = f.readlines()
    f.close()
with open('/content/drive/My Drive/ml/output/ucf101_3dcnnmodel_50_50_30_rgb.json', 'r') as f:
    model1 = model_from_json(f.read())
model1.load_weights('/content/drive/My Drive/ml/output/ucf101_3dcnnmodel_50_50_30_rgb.hd5', by_name=True)

# read video
video = '/content/drive/My Drive/ml/UCF101/ApplyEyeMakeup/v_ApplyEyeMakeup_g24_c02.avi'
cap = cv2.VideoCapture(video)

vid = []

while True:
    ret, frame = cap.read()
    #print("aaaaaaaaaaaaaa")
    if ret:
        #print("bbbbbbbbb")
        tmp=frame
       # tmp = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        vid.append(cv2.resize(tmp, (50,50)))
        if len(vid) == 30:
            i=np.array(vid)
           # print(i.shape)
           # j = np.transpose(i, (1,2,0))
            test = i.reshape((1,50, 50, 30, 3))
            pred = model1.predict(test)
            label = np.argmax(pred[0])
            print(label)
            cv2.putText(frame, class_names[label].split(' ')[-1].strip(), (20, 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        (0, 0, 255), 1)
            cv2.putText(frame, "prob: %.4f" % pred[0][label], (20, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        (0, 0, 255), 1)
            vid.pop(0)
        #cv2.imshow('result', frame)
        #cv2.waitKey(10)
    else:
        break
cap.release()
cv2.destroyAllWindows()


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
